# DGD+IOS算法+小值攻击

In [1]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [2]:
#定义变量 
agent_resource_quantity_robust1 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust1 = 0.6
dual_variable_robust1 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust1= 0.06 #100个用户中拜占庭用户的比例 100*proportion_robust1=拜占庭用户数量
iteration_robust1=[]#绘图横坐标
A_robust1 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵 对角元素为0
neighbor_robust1_help=[]
#q_robust1=[] #IOS鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量
W_robust1=np.loadtxt("100_Network_Topology_Adjacency_Matrix_B_0.0625(the value of bii=0.0625).txt") #双随机的网络拓扑矩阵W（对角线处非零）
a_robust1=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust1=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
check_robust1=np.zeros(shape=(100,60000))
q_robust1=4 #所有诚实用户邻居中拜占庭用户数量的上界

In [3]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust1*100)):
    dual_variable_robust1[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust1*100)),100):
    dual_variable_robust1[i][0]=-600
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust1*100)):
    agent_resource_quantity_robust1[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust1=0
for i in range (0,100-int(proportion_robust1*100)):
    sum_robust1 = sum_robust1 + agent_resource_quantity_robust1[i][0]
average_resource_honest_robust1[0][0]=sum_robust1/(100-int(proportion_robust1*100))

#诚实用户平均成本初始化
sum_robust1=0
for i in range(0,100-int(proportion_robust1*100)):
    sum_robust1=sum_robust1+a_robust1[i]*(agent_resource_quantity_robust1[i][0]-b_robust1[i])*(agent_resource_quantity_robust1[i][0]-b_robust1[i])
average_cost_honest_robust1[0][0]=sum_robust1/(100-int(proportion_robust1*100))

iteration_robust1.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust1[i][j]!=0:
            neighbor_robust1_help.append(j)
neighbor_robust1=np.array(neighbor_robust1_help).reshape(100,15)    

'''#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust1_help=0
    for j in range(0,15):
        if neighbor_robust1[i][j]==94 or neighbor_robust1[i][j]==95 or neighbor_robust1[i][j]==96 or neighbor_robust1[i][j]==97 or neighbor_robust1[i][j]==98 or neighbor_robust1[i][j]==99:
            q_robust1_help=q_robust1_help+1
    q_robust1.append(q_robust1_help)'''

'#计算诚实用户的拜占庭邻居用户数量\nfor i in range(0,100):\n    q_robust1_help=0\n    for j in range(0,15):\n        if neighbor_robust1[i][j]==94 or neighbor_robust1[i][j]==95 or neighbor_robust1[i][j]==96 or neighbor_robust1[i][j]==97 or neighbor_robust1[i][j]==98 or neighbor_robust1[i][j]==99:\n            q_robust1_help=q_robust1_help+1\n    q_robust1.append(q_robust1_help)'

In [4]:
#DGD+IOS算法 
def Robust_DGD_IOS_robust1(a_robust1,b_robust1,agent_resource_quantity_robust1,average_resource_honest_robust1,average_cost_honest_robust1,iteration_update_robust1,dual_variable_robust1,A_robust1,iteration_robust1,q_robust1):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+IOS+小值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust1[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust1[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust1[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust1[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust1)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust1.append(k)        
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust1=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust1*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust1.append(dual_variable_robust1[i][k-1]-iteration_update_robust1*((50/100)-(1/100)*agent_resource_quantity_robust1[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust1*100)),100):
            dual_variable_intermediate_robust1.append(dual_variable_robust1[i][k-1])
        #阶段二：IOS鲁棒聚合 每个用户聚合自身以及邻居的对偶值    
        for i in range (0,100-int(proportion_robust1*100)):
            u_neighbor_help_robust1=neighbor_robust1[i].tolist()#将用户i的邻居序号存在辅助变量u中
            u_neighbor_help_robust1.append(i)
            for q in range(0, int(q_robust1)):#剔除q个邻居的信息
                #计算当前的h-权重
                dual_variable_weight_average_robust1=0
                h_robust1=0
                for j in range(0,len(u_neighbor_help_robust1)):#计算当前循环中的h值
                    h_robust1=h_robust1+W_robust1[i][u_neighbor_help_robust1[j]]
                for j in range(0,len(u_neighbor_help_robust1)):#计算当前循环中的对偶权重平均值
                    dual_variable_weight_average_robust1=dual_variable_weight_average_robust1+(1/h_robust1)*(W_robust1[i][u_neighbor_help_robust1[j]]*dual_variable_intermediate_robust1[u_neighbor_help_robust1[j]])
                #剔除一个对偶值离权重平均对偶值最远的用户 保留自身的值
                u_neighbor_help_robust1.remove(i)
                neighbor_index_robust1=u_neighbor_help_robust1[0]
                max_distance_roubust1=abs(dual_variable_intermediate_robust1[u_neighbor_help_robust1[0]]-dual_variable_weight_average_robust1)
                for j in range(1,len(u_neighbor_help_robust1)):
                    if abs(dual_variable_intermediate_robust1[u_neighbor_help_robust1[j]]-dual_variable_weight_average_robust1)>=max_distance_roubust1:
                        max_distance_roubust1=abs(dual_variable_intermediate_robust1[u_neighbor_help_robust1[j]]-dual_variable_weight_average_robust1)
                        neighbor_index_robust1=u_neighbor_help_robust1[j]
                u_neighbor_help_robust1.remove(neighbor_index_robust1)
                u_neighbor_help_robust1.append(i)
            #计算剔除q个信息后的权重平均对偶值
            dual_variable_weight_average_robust1=0
            h_robust1=0
            for j in range(0,len(u_neighbor_help_robust1)):
                h_robust1=h_robust1+W_robust1[i][u_neighbor_help_robust1[j]]
            for j in range(0,len(u_neighbor_help_robust1)):
                dual_variable_weight_average_robust1=dual_variable_weight_average_robust1+(1/h_robust1)*(W_robust1[i][u_neighbor_help_robust1[j]]*dual_variable_intermediate_robust1[u_neighbor_help_robust1[j]])     
            #诚实用户基于IOS鲁棒聚合后的值进行自身的对偶变量的更新 基于权重平均值
            dual_variable_robust1[i][k]=dual_variable_weight_average_robust1
            
            '''#检测是否将拜占庭用户剔除
            for j in range(0,len(u_neighbor_help_robust1)):
                if u_neighbor_help_robust1[j]==94 or u_neighbor_help_robust1[j]==95 or u_neighbor_help_robust1[j]==96 or u_neighbor_help_robust1[j]==97 or u_neighbor_help_robust1[j]==98 or u_neighbor_help_robust1[j]==99:
                    check_robust1[i][k]=1 '''
            
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust1*100),100):
            dual_variable_robust1[i][k]=dual_variable_robust1[i][k-1]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust1*100)):
            agent_resource_quantity_robust1[i][k]=(dual_variable_robust1[i][k]-2*a_robust1[i]*b_robust1[i])/(-2*a_robust1[i])
            #原变量的投影操作
            if agent_resource_quantity_robust1[i][k] < 0:
                agent_resource_quantity_robust1[i][k]=0
            elif agent_resource_quantity_robust1[i][k] > 100: 
                agent_resource_quantity_robust1[i][k]=100  
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust1*100),100):
            agent_resource_quantity_robust1[i][k]=agent_resource_quantity_robust1[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust1=0
        for i in range (0,100-int(proportion_robust1*100)):
            sum1_robust1 = sum1_robust1 + agent_resource_quantity_robust1[i][k]
        average_resource_honest_robust1[0][k]=sum1_robust1/(100-int(proportion_robust1*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust1=0
        for i in range(0,100-int(proportion_robust1*100)):
            sum2_robust1=sum2_robust1+a_robust1[i]*(agent_resource_quantity_robust1[i][k]-b_robust1[i])*(agent_resource_quantity_robust1[i][k]-b_robust1[i])
        average_cost_honest_robust1[0][k]=sum2_robust1/(100-int(proportion_robust1*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust1[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust1[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust1[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust1[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust1)
            print("****************************************************************************")
            '''print("聚合权重为:",h_robust1)
            print("****************************************************************************")'''
        
        iteration_update_robust1=1/(pow(k+1,0.1)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [5]:
Robust_DGD_IOS_robust1(a_robust1,b_robust1,agent_resource_quantity_robust1,average_resource_honest_robust1,average_cost_honest_robust1,iteration_update_robust1,dual_variable_robust1,A_robust1,iteration_robust1,q_robust1)

****************************DGD算法+IOS+小值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 56.57291376360089
agent 2 在 3999 次迭代中的资源申请量为: 56.39722173349602
agent 3 在 3999 次迭代中的资源申请量为: 61.08302575264673
agent 4 在 3999 次迭代中的资源申请量为: 38.1867233654013
agent 5 在 3999 次迭代中的资源申请量为: 57.9518291541347
agent 6 在 3999 次迭代中的资源申请量为: 55.73321685123774
agent 7 在 3999 次迭代中的资源申请量为: 66.09588984297694
agent 8 在 3999 次迭代中的资源申请量为: 46.671782714045634
agent 9 在 3999 次迭代中的资源申请量为: 66.08514772481054
agent 10 在 3999 次迭代中的资源申请量为: 54.584260025154705
agent 11 在 3999 次迭代中的资源申请量为: 65.90964109257676
agent 12 在 3999 次迭代中的资源申请量为: 38.683431365494826
agent 13 在 3999 次迭代中的资源申请量为: 50.06065594489135
agent 14 在 3999 次迭代中的资源申请量为: 41.64633511029743
agent 15 在 3999 次迭代中的资源申请量为: 42.79281860110122
agent 16 在 3999 次迭代中的资源申请量为: 49.594885739619585
agent 17 在 3999 次迭代中的资源申请量为: 50.45120133075163
agent 18 在 3999 次迭代中的资源申请量为: 46.881645983260334
agent 19 在 3999 次迭代中的资源申请量为: 45.110719495772784
agent 20 在 3999 次迭代中的资源申请量为: 51.14133719819595
agent 21 在 3999 次迭代中的资源申请量为: 70.3994138834136
agent

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 56.39244423352228
agent 2 在 5999 次迭代中的资源申请量为: 56.219246374582
agent 3 在 5999 次迭代中的资源申请量为: 60.884137963983456
agent 4 在 5999 次迭代中的资源申请量为: 38.06483723432002
agent 5 在 5999 次迭代中的资源申请量为: 57.76824663821338
agent 6 在 5999 次迭代中的资源申请量为: 55.55592443461221
agent 7 在 5999 次迭代中的资源申请量为: 65.87969211990644
agent 8 在 5999 次迭代中的资源申请量为: 46.522445091576465
agent 9 在 5999 次迭代中的资源申请量为: 65.87914195356397
agent 10 在 5999 次迭代中的资源申请量为: 54.41157829843846
agent 11 在 5999 次迭代中的资源申请量为: 65.69384078491187
agent 12 在 5999 次迭代中的资源申请量为: 38.55980546811647
agent 13 在 5999 次迭代中的资源申请量为: 49.90024601637019
agent 14 在 5999 次迭代中的资源申请量为: 41.51443569293387
agent 15 在 5999 次迭代中的资源申请量为: 42.656818447921076
agent 16 在 5999 次迭代中的资源申请量为: 49.43650998553947
agent 17 在 5999 次迭代中的资源申请量为: 50.292357203575435
agent 18 在 5999 次迭代中的资源申请量为: 46.7307999272165
agent 19 在 5999 次迭代中的资源申请量为: 44.96358998887069
agent 20 在 5999 次迭代中的资源申请量为: 50.98029608895721
agent 21 在 5999 次迭代中的资源申请量为: 70.17392755482683
agent 2

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 56.37872558404526
agent 2 在 7999 次迭代中的资源申请量为: 56.20560157245145
agent 3 在 7999 次迭代中的资源申请量为: 60.868238079293256
agent 4 在 7999 次迭代中的资源申请量为: 38.05511942210575
agent 5 在 7999 次迭代中的资源申请量为: 57.75332302682588
agent 6 在 7999 次迭代中的资源申请量为: 55.54152168828745
agent 7 在 7999 次迭代中的资源申请量为: 65.86234771616884
agent 8 在 7999 次迭代中的资源申请量为: 46.509774522639724
agent 9 在 7999 次迭代中的资源申请量为: 65.85598592813955
agent 10 在 7999 次迭代中的资源申请量为: 54.39792596115451
agent 11 在 7999 次迭代中的资源申请量为: 65.67647587295149
agent 12 在 7999 次迭代中的资源申请量为: 38.549456166674304
agent 13 在 7999 次迭代中的资源申请量为: 49.88684161155929
agent 14 在 7999 次迭代中的资源申请量为: 41.50389608191523
agent 15 在 7999 次迭代中的资源申请量为: 42.64569853493217
agent 16 在 7999 次迭代中的资源申请量为: 49.42367418645401
agent 17 在 7999 次迭代中的资源申请量为: 50.27945350516737
agent 18 在 7999 次迭代中的资源申请量为: 46.718339998030046
agent 19 在 7999 次迭代中的资源申请量为: 44.951130199398385
agent 20 在 7999 次迭代中的资源申请量为: 50.967698741811894
agent 21 在 7999 次迭代中的资源申请量为: 70.15575997016403
ag

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 56.37587053242325
agent 2 在 9999 次迭代中的资源申请量为: 56.202796702780965
agent 3 在 9999 次迭代中的资源申请量为: 60.8648294640941
agent 4 在 9999 次迭代中的资源申请量为: 38.05329459917122
agent 5 在 9999 次迭代中的资源申请量为: 57.75017679188492
agent 6 在 9999 次迭代中的资源申请量为: 55.5378218498249
agent 7 在 9999 次迭代中的资源申请量为: 65.85820923020168
agent 8 在 9999 次迭代中的资源申请量为: 46.5068345035661
agent 9 在 9999 次迭代中的资源申请量为: 65.85250802838203
agent 10 在 9999 次迭代中的资源申请量为: 54.39503545975119
agent 11 在 9999 次迭代中的资源申请量为: 65.67192011079423
agent 12 在 9999 次迭代中的资源申请量为: 38.547194103362536
agent 13 在 9999 次迭代中的资源申请量为: 49.880433334976
agent 14 在 9999 次迭代中的资源申请量为: 41.5023429138488
agent 15 在 9999 次迭代中的资源申请量为: 42.64293900714455
agent 16 在 9999 次迭代中的资源申请量为: 49.42063803384619
agent 17 在 9999 次迭代中的资源申请量为: 50.27715731976509
agent 18 在 9999 次迭代中的资源申请量为: 46.715432644534246
agent 19 在 9999 次迭代中的资源申请量为: 44.948210568911435
agent 20 在 9999 次迭代中的资源申请量为: 50.964934129401506
agent 21 在 9999 次迭代中的资源申请量为: 70.15261823916069
agent 22 

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 56.37758839171811
agent 2 在 51999 次迭代中的资源申请量为: 56.206275261234865
agent 3 在 51999 次迭代中的资源申请量为: 60.86324527198883
agent 4 在 51999 次迭代中的资源申请量为: 38.05444965605938
agent 5 在 51999 次迭代中的资源申请量为: 57.75159408523659
agent 6 在 51999 次迭代中的资源申请量为: 55.53615705154922
agent 7 在 51999 次迭代中的资源申请量为: 65.85676850598307
agent 8 在 51999 次迭代中的资源申请量为: 46.50363487086908
agent 9 在 51999 次迭代中的资源申请量为: 65.85440683132869
agent 10 在 51999 次迭代中的资源申请量为: 54.399058004599716
agent 11 在 51999 次迭代中的资源申请量为: 65.67294324600513
agent 12 在 51999 次迭代中的资源申请量为: 38.54527252937365
agent 13 在 51999 次迭代中的资源申请量为: 49.87900577695267
agent 14 在 51999 次迭代中的资源申请量为: 41.502432281344014
agent 15 在 51999 次迭代中的资源申请量为: 42.64094023340903
agent 16 在 51999 次迭代中的资源申请量为: 49.41901507201586
agent 17 在 51999 次迭代中的资源申请量为: 50.27830091877743
agent 18 在 51999 次迭代中的资源申请量为: 46.71205919888747
agent 19 在 51999 次迭代中的资源申请量为: 44.94489055599918
agent 20 在 51999 次迭代中的资源申请量为: 50.9659670691321
agent 21 在 51999 次迭代中的资源申请量为: 70

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 56.37727335814798
agent 2 在 53999 次迭代中的资源申请量为: 56.20649222079688
agent 3 在 53999 次迭代中的资源申请量为: 60.862781058031615
agent 4 在 53999 次迭代中的资源申请量为: 38.05449040056214
agent 5 在 53999 次迭代中的资源申请量为: 57.75207560571343
agent 6 在 53999 次迭代中的资源申请量为: 55.53633152944671
agent 7 在 53999 次迭代中的资源申请量为: 65.85669651372427
agent 8 在 53999 次迭代中的资源申请量为: 46.503593732041075
agent 9 在 53999 次迭代中的资源申请量为: 65.85967555043514
agent 10 在 53999 次迭代中的资源申请量为: 54.39985848116011
agent 11 在 53999 次迭代中的资源申请量为: 65.67384208200173
agent 12 在 53999 次迭代中的资源申请量为: 38.54519353793994
agent 13 在 53999 次迭代中的资源申请量为: 49.88169407878632
agent 14 在 53999 次迭代中的资源申请量为: 41.50189343195017
agent 15 在 53999 次迭代中的资源申请量为: 42.64088723060619
agent 16 在 53999 次迭代中的资源申请量为: 49.41891912107662
agent 17 在 53999 次迭代中的资源申请量为: 50.278377463316865
agent 18 在 53999 次迭代中的资源申请量为: 46.71168879067632
agent 19 在 53999 次迭代中的资源申请量为: 44.94480756694356
agent 20 在 53999 次迭代中的资源申请量为: 50.96597594715283
agent 21 在 53999 次迭代中的资源申请量为: 7

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 56.377671067410056
agent 2 在 55999 次迭代中的资源申请量为: 56.20662374710079
agent 3 在 55999 次迭代中的资源申请量为: 60.86299885203942
agent 4 在 55999 次迭代中的资源申请量为: 38.0544150454895
agent 5 在 55999 次迭代中的资源申请量为: 57.751810644101894
agent 6 在 55999 次迭代中的资源申请量为: 55.53643132718092
agent 7 在 55999 次迭代中的资源申请量为: 65.85689468788337
agent 8 在 55999 次迭代中的资源申请量为: 46.50343678754689
agent 9 在 55999 次迭代中的资源申请量为: 65.85457268260036
agent 10 在 55999 次迭代中的资源申请量为: 54.39962845549212
agent 11 在 55999 次迭代中的资源申请量为: 65.67379526831851
agent 12 在 55999 次迭代中的资源申请量为: 38.5450840473914
agent 13 在 55999 次迭代中的资源申请量为: 49.88174118147473
agent 14 在 55999 次迭代中的资源申请量为: 41.50187261780079
agent 15 在 55999 次迭代中的资源申请量为: 42.64097392111692
agent 16 在 55999 次迭代中的资源申请量为: 49.41902136060781
agent 17 在 55999 次迭代中的资源申请量为: 50.27813018437617
agent 18 在 55999 次迭代中的资源申请量为: 46.71179100989178
agent 19 在 55999 次迭代中的资源申请量为: 44.94468840019774
agent 20 在 55999 次迭代中的资源申请量为: 50.96613445704986
agent 21 在 55999 次迭代中的资源申请量为: 70.1

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 56.377704332376
agent 2 在 57999 次迭代中的资源申请量为: 56.206497808189305
agent 3 在 57999 次迭代中的资源申请量为: 60.86316221641174
agent 4 在 57999 次迭代中的资源申请量为: 38.054527652552146
agent 5 在 57999 次迭代中的资源申请量为: 57.751692072278104
agent 6 在 57999 次迭代中的资源申请量为: 55.53606771647086
agent 7 在 57999 次迭代中的资源申请量为: 65.85669528348942
agent 8 在 57999 次迭代中的资源申请量为: 46.50345066725779
agent 9 在 57999 次迭代中的资源申请量为: 65.8545358763489
agent 10 在 57999 次迭代中的资源申请量为: 54.399313241773335
agent 11 在 57999 次迭代中的资源申请量为: 65.67301948408095
agent 12 在 57999 次迭代中的资源申请量为: 38.545164121939166
agent 13 在 57999 次迭代中的资源申请量为: 49.87892993968458
agent 14 在 57999 次迭代中的资源申请量为: 41.50244628948599
agent 15 在 57999 次迭代中的资源申请量为: 42.64082795933531
agent 16 在 57999 次迭代中的资源申请量为: 49.418927021637614
agent 17 在 57999 次迭代中的资源申请量为: 50.278380655626904
agent 18 在 57999 次迭代中的资源申请量为: 46.711864516741734
agent 19 在 57999 次迭代中的资源申请量为: 44.944698903213315
agent 20 在 57999 次迭代中的资源申请量为: 50.96604022350041
agent 21 在 57999 次迭代中的资源申请量为

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 56.3773885359945
agent 2 在 59999 次迭代中的资源申请量为: 56.20670471401895
agent 3 在 59999 次迭代中的资源申请量为: 60.86270576327959
agent 4 在 59999 次迭代中的资源申请量为: 38.05456524759511
agent 5 在 59999 次迭代中的资源申请量为: 57.75216512558106
agent 6 在 59999 次迭代中的资源申请量为: 55.53624349344336
agent 7 在 59999 次迭代中的资源申请量为: 65.85662660233905
agent 8 在 59999 次迭代中的资源申请量为: 46.50341640346253
agent 9 在 59999 次迭代中的资源申请量为: 65.85974500722061
agent 10 在 59999 次迭代中的资源申请量为: 54.400096430280875
agent 11 在 59999 次迭代中的资源申请量为: 65.67390625753639
agent 12 在 59999 次迭代中的资源申请量为: 38.545089748602464
agent 13 在 59999 次迭代中的资源申请量为: 49.88159271395787
agent 14 在 59999 次迭代中的资源申请量为: 41.50191258561869
agent 15 在 59999 次迭代中的资源申请量为: 42.64077943771322
agent 16 在 59999 次迭代中的资源申请量为: 49.418835153380385
agent 17 在 59999 次迭代中的资源申请量为: 50.27845361369593
agent 18 在 59999 次迭代中的资源申请量为: 46.7115047912937
agent 19 在 59999 次迭代中的资源申请量为: 44.944623484711926
agent 20 在 59999 次迭代中的资源申请量为: 50.966046448768594
agent 21 在 59999 次迭代中的资源申请量为: 7

In [6]:
np.savetxt('./data/01_IOS_q=4_agent_resource_quantity_min_attack.txt',agent_resource_quantity_robust1,fmt='%0.8f')#用户资源
np.savetxt('./data/01_IOS_q=4_average_cost_honest_min_attack.txt',average_cost_honest_robust1,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/01_IOS_q=4_average_resource_honest_min_attack.txt',average_resource_honest_robust1,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/01_IOS_q=4_dual_variable_min_attack.txt',dual_variable_robust1,fmt='%0.8f')#用户对偶变量

# DGD+IOS算法+大值攻击

In [7]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [8]:
#定义变量 
agent_resource_quantity_robust2 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust2 = 0.6
dual_variable_robust2 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust2= 0.06 #100个用户中拜占庭用户的比例 100*proportion_robust2=拜占庭用户数量
iteration_robust2=[]#绘图横坐标
A_robust2 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵
neighbor_robust2_help=[]
q_robust2=[] #IOS鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量
W_robust2=np.loadtxt("100_Network_Topology_Adjacency_Matrix_B_0.0625(the value of bii=0.0625).txt") #双随机的网络拓扑矩阵W（对角线处非零）
a_robust2=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust2=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
check_robust2=np.zeros(shape=(100,60000))
q_robust2=4 #所有诚实用户邻居中拜占庭用户数量的上界

In [9]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust2*100)):
    dual_variable_robust2[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust2*100)),100):
    dual_variable_robust2[i][0]=-0.01
      
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust2*100)):
    agent_resource_quantity_robust2[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust2=0
for i in range (0,100-int(proportion_robust2*100)):
    sum_robust2 = sum_robust2 + agent_resource_quantity_robust2[i][0]
average_resource_honest_robust2[0][0]=sum_robust2/(100-int(proportion_robust2*100))

#诚实用户平均成本初始化
sum_robust2=0
for i in range(0,100-int(proportion_robust2*100)):
    sum_robust2=sum_robust2+a_robust2[i]*(agent_resource_quantity_robust2[i][0]-b_robust2[i])*(agent_resource_quantity_robust2[i][0]-b_robust2[i])
average_cost_honest_robust2[0][0]=sum_robust2/(100-int(proportion_robust2*100))

iteration_robust2.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust2[i][j]!=0:
            neighbor_robust2_help.append(j)
neighbor_robust2=np.array(neighbor_robust2_help).reshape(100,15)

'''#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust2_help=0
    for j in range(0,15):
        if neighbor_robust2[i][j]==94 or neighbor_robust2[i][j]==95 or neighbor_robust2[i][j]==96 or neighbor_robust2[i][j]==97 or neighbor_robust2[i][j]==98 or neighbor_robust2[i][j]==99:
            q_robust2_help=q_robust2_help+1
    q_robust2.append(q_robust2_help) '''

'#计算诚实用户的拜占庭邻居用户数量\nfor i in range(0,100):\n    q_robust2_help=0\n    for j in range(0,15):\n        if neighbor_robust2[i][j]==94 or neighbor_robust2[i][j]==95 or neighbor_robust2[i][j]==96 or neighbor_robust2[i][j]==97 or neighbor_robust2[i][j]==98 or neighbor_robust2[i][j]==99:\n            q_robust2_help=q_robust2_help+1\n    q_robust2.append(q_robust2_help) '

In [10]:
#DGD+IOS算法 
def Robust_DGD_IOS_robust2(a_robust2,b_robust2,agent_resource_quantity_robust2,average_resource_honest_robust2,average_cost_honest_robust2,iteration_update_robust2,dual_variable_robust2,A_robust2,iteration_robust2,q_robust2):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+IOS+大值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust2[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust2[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust2[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust2[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust2)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust2.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust2=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust2*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust2.append(dual_variable_robust2[i][k-1]-iteration_update_robust2*((50/100)-(1/100)*agent_resource_quantity_robust2[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust2*100)),100):
            dual_variable_intermediate_robust2.append(dual_variable_robust2[i][k-1])
        #阶段二：IOS鲁棒聚合 每个用户聚合自身以及邻居的对偶值    
        for i in range (0,100-int(proportion_robust2*100)):
            u_neighbor_help_robust2=neighbor_robust2[i].tolist()#将用户i的邻居序号存在辅助变量u中
            u_neighbor_help_robust2.append(i)
            for q in range(0, int(q_robust2)):#剔除q个邻居的信息
                #计算当前的h-权重
                dual_variable_weight_average_robust2=0
                h_robust2=0
                for j in range(0,len(u_neighbor_help_robust2)):#计算当前循环中的h值
                    h_robust2=h_robust2+W_robust2[i][u_neighbor_help_robust2[j]]
                for j in range(0,len(u_neighbor_help_robust2)):#计算当前循环中的对偶权重平均值
                    dual_variable_weight_average_robust2=dual_variable_weight_average_robust2+(1/h_robust2)*(W_robust2[i][u_neighbor_help_robust2[j]]*dual_variable_intermediate_robust2[u_neighbor_help_robust2[j]])
                #剔除一个对偶值离权重平均对偶值最远的用户 保留自身的值
                u_neighbor_help_robust2.remove(i)
                neighbor_index_robust2=u_neighbor_help_robust2[0]
                max_distance_roubust2=abs(dual_variable_intermediate_robust2[u_neighbor_help_robust2[0]]-dual_variable_weight_average_robust2)
                for j in range(1,len(u_neighbor_help_robust2)):
                    if abs(dual_variable_intermediate_robust2[u_neighbor_help_robust2[j]]-dual_variable_weight_average_robust2)>=max_distance_roubust2:
                        max_distance_roubust2=abs(dual_variable_intermediate_robust2[u_neighbor_help_robust2[j]]-dual_variable_weight_average_robust2)
                        neighbor_index_robust2=u_neighbor_help_robust2[j]
                u_neighbor_help_robust2.remove(neighbor_index_robust2)
                u_neighbor_help_robust2.append(i)
            #计算剔除q个信息后的权重平均对偶值
            dual_variable_weight_average_robust2=0
            h_robust2=0
            for j in range(0,len(u_neighbor_help_robust2)):
                h_robust2=h_robust2+W_robust2[i][u_neighbor_help_robust2[j]]
            for j in range(0,len(u_neighbor_help_robust2)):
                dual_variable_weight_average_robust2=dual_variable_weight_average_robust2+(1/h_robust2)*(W_robust2[i][u_neighbor_help_robust2[j]]*dual_variable_intermediate_robust2[u_neighbor_help_robust2[j]])     
            #诚实用户基于IOS鲁棒聚合后的值进行自身的对偶变量的更新 基于权重平均值
            dual_variable_robust2[i][k]=dual_variable_weight_average_robust2
            
            '''#检测是否将拜占庭用户剔除
            for j in range(0,len(u_neighbor_help_robust2)):
                if u_neighbor_help_robust2[j]==94 or u_neighbor_help_robust2[j]==95 or u_neighbor_help_robust2[j]==96 or u_neighbor_help_robust2[j]==97 or u_neighbor_help_robust2[j]==98 or u_neighbor_help_robust2[j]==99:
                    check_robust2[i][k]=1'''
        
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust2*100),100):
            dual_variable_robust2[i][k]=dual_variable_robust2[i][k-1]            
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust2*100)):
            agent_resource_quantity_robust2[i][k]=(dual_variable_robust2[i][k]-2*a_robust2[i]*b_robust2[i])/(-2*a_robust2[i])
            #原变量的投影操作
            if agent_resource_quantity_robust2[i][k] < 0:
                agent_resource_quantity_robust2[i][k]=0
            elif agent_resource_quantity_robust2[i][k] > 100: 
                agent_resource_quantity_robust2[i][k]=100   
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust2*100),100):
            agent_resource_quantity_robust2[i][k]=agent_resource_quantity_robust2[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust2=0
        for i in range (0,100-int(proportion_robust2*100)):
            sum1_robust2 = sum1_robust2 + agent_resource_quantity_robust2[i][k]
        average_resource_honest_robust2[0][k]=sum1_robust2/(100-int(proportion_robust2*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust2=0
        for i in range(0,100-int(proportion_robust2*100)):
            sum2_robust2=sum2_robust2+a_robust2[i]*(agent_resource_quantity_robust2[i][k]-b_robust2[i])*(agent_resource_quantity_robust2[i][k]-b_robust2[i])
        average_cost_honest_robust2[0][k]=sum2_robust2/(100-int(proportion_robust2*100))
        
       #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust2[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust2[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust2[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust2[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust2)
            print("****************************************************************************")
        
        iteration_update_robust2=1/(pow(k+1,0.1)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [11]:
Robust_DGD_IOS_robust2(a_robust2,b_robust2,agent_resource_quantity_robust2,average_resource_honest_robust2,average_cost_honest_robust2,iteration_update_robust2,dual_variable_robust2,A_robust2,iteration_robust2,q_robust2)

****************************DGD算法+IOS+大值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 56.57291376360089
agent 2 在 3999 次迭代中的资源申请量为: 56.39722173349602
agent 3 在 3999 次迭代中的资源申请量为: 61.08302575264673
agent 4 在 3999 次迭代中的资源申请量为: 38.1867233654013
agent 5 在 3999 次迭代中的资源申请量为: 57.9518291541347
agent 6 在 3999 次迭代中的资源申请量为: 55.73321685123774
agent 7 在 3999 次迭代中的资源申请量为: 66.09588984297694
agent 8 在 3999 次迭代中的资源申请量为: 46.671782714045634
agent 9 在 3999 次迭代中的资源申请量为: 66.08514772481054
agent 10 在 3999 次迭代中的资源申请量为: 54.584260025154705
agent 11 在 3999 次迭代中的资源申请量为: 65.90964109257676
agent 12 在 3999 次迭代中的资源申请量为: 38.683431365494826
agent 13 在 3999 次迭代中的资源申请量为: 50.06065594489135
agent 14 在 3999 次迭代中的资源申请量为: 41.64633511029743
agent 15 在 3999 次迭代中的资源申请量为: 42.79281860110122
agent 16 在 3999 次迭代中的资源申请量为: 49.594885739619585
agent 17 在 3999 次迭代中的资源申请量为: 50.45120133075163
agent 18 在 3999 次迭代中的资源申请量为: 46.881645983260334
agent 19 在 3999 次迭代中的资源申请量为: 45.110719495772784
agent 20 在 3999 次迭代中的资源申请量为: 51.14133719819595
agent 21 在 3999 次迭代中的资源申请量为: 70.3994138834136
agent

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 56.39244423352228
agent 2 在 5999 次迭代中的资源申请量为: 56.219246374582
agent 3 在 5999 次迭代中的资源申请量为: 60.884137963983456
agent 4 在 5999 次迭代中的资源申请量为: 38.06483723432002
agent 5 在 5999 次迭代中的资源申请量为: 57.76824663821338
agent 6 在 5999 次迭代中的资源申请量为: 55.55592443461221
agent 7 在 5999 次迭代中的资源申请量为: 65.87969211990644
agent 8 在 5999 次迭代中的资源申请量为: 46.522445091576465
agent 9 在 5999 次迭代中的资源申请量为: 65.87914195356397
agent 10 在 5999 次迭代中的资源申请量为: 54.41157829843846
agent 11 在 5999 次迭代中的资源申请量为: 65.69384078491187
agent 12 在 5999 次迭代中的资源申请量为: 38.55980546811647
agent 13 在 5999 次迭代中的资源申请量为: 49.90024601637019
agent 14 在 5999 次迭代中的资源申请量为: 41.51443569293387
agent 15 在 5999 次迭代中的资源申请量为: 42.656818447921076
agent 16 在 5999 次迭代中的资源申请量为: 49.43650998553947
agent 17 在 5999 次迭代中的资源申请量为: 50.292357203575435
agent 18 在 5999 次迭代中的资源申请量为: 46.7307999272165
agent 19 在 5999 次迭代中的资源申请量为: 44.96358998887069
agent 20 在 5999 次迭代中的资源申请量为: 50.98029608895721
agent 21 在 5999 次迭代中的资源申请量为: 70.17392755482683
agent 2

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 56.37872558404526
agent 2 在 7999 次迭代中的资源申请量为: 56.20560157245145
agent 3 在 7999 次迭代中的资源申请量为: 60.868238079293256
agent 4 在 7999 次迭代中的资源申请量为: 38.05511942210575
agent 5 在 7999 次迭代中的资源申请量为: 57.75332302682588
agent 6 在 7999 次迭代中的资源申请量为: 55.54152168828745
agent 7 在 7999 次迭代中的资源申请量为: 65.86234771616884
agent 8 在 7999 次迭代中的资源申请量为: 46.509774522639724
agent 9 在 7999 次迭代中的资源申请量为: 65.85598592813955
agent 10 在 7999 次迭代中的资源申请量为: 54.39792596115451
agent 11 在 7999 次迭代中的资源申请量为: 65.67647587295149
agent 12 在 7999 次迭代中的资源申请量为: 38.549456166674304
agent 13 在 7999 次迭代中的资源申请量为: 49.88684161155929
agent 14 在 7999 次迭代中的资源申请量为: 41.50389608191523
agent 15 在 7999 次迭代中的资源申请量为: 42.64569853493217
agent 16 在 7999 次迭代中的资源申请量为: 49.42367418645401
agent 17 在 7999 次迭代中的资源申请量为: 50.27945350516737
agent 18 在 7999 次迭代中的资源申请量为: 46.718339998030046
agent 19 在 7999 次迭代中的资源申请量为: 44.951130199398385
agent 20 在 7999 次迭代中的资源申请量为: 50.967698741811894
agent 21 在 7999 次迭代中的资源申请量为: 70.15575997016403
ag

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 56.37587053242325
agent 2 在 9999 次迭代中的资源申请量为: 56.202796702780965
agent 3 在 9999 次迭代中的资源申请量为: 60.8648294640941
agent 4 在 9999 次迭代中的资源申请量为: 38.05329459917122
agent 5 在 9999 次迭代中的资源申请量为: 57.75017679188492
agent 6 在 9999 次迭代中的资源申请量为: 55.5378218498249
agent 7 在 9999 次迭代中的资源申请量为: 65.85820923020168
agent 8 在 9999 次迭代中的资源申请量为: 46.5068345035661
agent 9 在 9999 次迭代中的资源申请量为: 65.85250802838203
agent 10 在 9999 次迭代中的资源申请量为: 54.39503545975119
agent 11 在 9999 次迭代中的资源申请量为: 65.67192011079423
agent 12 在 9999 次迭代中的资源申请量为: 38.547194103362536
agent 13 在 9999 次迭代中的资源申请量为: 49.880433334976
agent 14 在 9999 次迭代中的资源申请量为: 41.5023429138488
agent 15 在 9999 次迭代中的资源申请量为: 42.64293900714455
agent 16 在 9999 次迭代中的资源申请量为: 49.42063803384619
agent 17 在 9999 次迭代中的资源申请量为: 50.27715731976509
agent 18 在 9999 次迭代中的资源申请量为: 46.715432644534246
agent 19 在 9999 次迭代中的资源申请量为: 44.948210568911435
agent 20 在 9999 次迭代中的资源申请量为: 50.964934129401506
agent 21 在 9999 次迭代中的资源申请量为: 70.15261823916069
agent 22 

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 56.37758839171811
agent 2 在 51999 次迭代中的资源申请量为: 56.206275261234865
agent 3 在 51999 次迭代中的资源申请量为: 60.86324527198883
agent 4 在 51999 次迭代中的资源申请量为: 38.05444965605938
agent 5 在 51999 次迭代中的资源申请量为: 57.75159408523659
agent 6 在 51999 次迭代中的资源申请量为: 55.53615705154922
agent 7 在 51999 次迭代中的资源申请量为: 65.85676850598307
agent 8 在 51999 次迭代中的资源申请量为: 46.50363487086908
agent 9 在 51999 次迭代中的资源申请量为: 65.85440683132869
agent 10 在 51999 次迭代中的资源申请量为: 54.399058004599716
agent 11 在 51999 次迭代中的资源申请量为: 65.67294324600513
agent 12 在 51999 次迭代中的资源申请量为: 38.54527252937365
agent 13 在 51999 次迭代中的资源申请量为: 49.87900577695267
agent 14 在 51999 次迭代中的资源申请量为: 41.502432281344014
agent 15 在 51999 次迭代中的资源申请量为: 42.64094023340903
agent 16 在 51999 次迭代中的资源申请量为: 49.41901507201586
agent 17 在 51999 次迭代中的资源申请量为: 50.27830091877743
agent 18 在 51999 次迭代中的资源申请量为: 46.71205919888747
agent 19 在 51999 次迭代中的资源申请量为: 44.94489055599918
agent 20 在 51999 次迭代中的资源申请量为: 50.9659670691321
agent 21 在 51999 次迭代中的资源申请量为: 70

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 56.37727335814798
agent 2 在 53999 次迭代中的资源申请量为: 56.20649222079688
agent 3 在 53999 次迭代中的资源申请量为: 60.862781058031615
agent 4 在 53999 次迭代中的资源申请量为: 38.05449040056214
agent 5 在 53999 次迭代中的资源申请量为: 57.75207560571343
agent 6 在 53999 次迭代中的资源申请量为: 55.53633152944671
agent 7 在 53999 次迭代中的资源申请量为: 65.85669651372427
agent 8 在 53999 次迭代中的资源申请量为: 46.503593732041075
agent 9 在 53999 次迭代中的资源申请量为: 65.85967555043514
agent 10 在 53999 次迭代中的资源申请量为: 54.39985848116011
agent 11 在 53999 次迭代中的资源申请量为: 65.67384208200173
agent 12 在 53999 次迭代中的资源申请量为: 38.54519353793994
agent 13 在 53999 次迭代中的资源申请量为: 49.88169407878632
agent 14 在 53999 次迭代中的资源申请量为: 41.50189343195017
agent 15 在 53999 次迭代中的资源申请量为: 42.64088723060619
agent 16 在 53999 次迭代中的资源申请量为: 49.41891912107662
agent 17 在 53999 次迭代中的资源申请量为: 50.278377463316865
agent 18 在 53999 次迭代中的资源申请量为: 46.71168879067632
agent 19 在 53999 次迭代中的资源申请量为: 44.94480756694356
agent 20 在 53999 次迭代中的资源申请量为: 50.96597594715283
agent 21 在 53999 次迭代中的资源申请量为: 7

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 56.377671067410056
agent 2 在 55999 次迭代中的资源申请量为: 56.20662374710079
agent 3 在 55999 次迭代中的资源申请量为: 60.86299885203942
agent 4 在 55999 次迭代中的资源申请量为: 38.0544150454895
agent 5 在 55999 次迭代中的资源申请量为: 57.751810644101894
agent 6 在 55999 次迭代中的资源申请量为: 55.53643132718092
agent 7 在 55999 次迭代中的资源申请量为: 65.85689468788337
agent 8 在 55999 次迭代中的资源申请量为: 46.50343678754689
agent 9 在 55999 次迭代中的资源申请量为: 65.85457268260036
agent 10 在 55999 次迭代中的资源申请量为: 54.39962845549212
agent 11 在 55999 次迭代中的资源申请量为: 65.67379526831851
agent 12 在 55999 次迭代中的资源申请量为: 38.5450840473914
agent 13 在 55999 次迭代中的资源申请量为: 49.88174118147473
agent 14 在 55999 次迭代中的资源申请量为: 41.50187261780079
agent 15 在 55999 次迭代中的资源申请量为: 42.64097392111692
agent 16 在 55999 次迭代中的资源申请量为: 49.41902136060781
agent 17 在 55999 次迭代中的资源申请量为: 50.27813018437617
agent 18 在 55999 次迭代中的资源申请量为: 46.71179100989178
agent 19 在 55999 次迭代中的资源申请量为: 44.94468840019774
agent 20 在 55999 次迭代中的资源申请量为: 50.96613445704986
agent 21 在 55999 次迭代中的资源申请量为: 70.1

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 56.377704332376
agent 2 在 57999 次迭代中的资源申请量为: 56.206497808189305
agent 3 在 57999 次迭代中的资源申请量为: 60.86316221641174
agent 4 在 57999 次迭代中的资源申请量为: 38.054527652552146
agent 5 在 57999 次迭代中的资源申请量为: 57.751692072278104
agent 6 在 57999 次迭代中的资源申请量为: 55.53606771647086
agent 7 在 57999 次迭代中的资源申请量为: 65.85669528348942
agent 8 在 57999 次迭代中的资源申请量为: 46.50345066725779
agent 9 在 57999 次迭代中的资源申请量为: 65.8545358763489
agent 10 在 57999 次迭代中的资源申请量为: 54.399313241773335
agent 11 在 57999 次迭代中的资源申请量为: 65.67301948408095
agent 12 在 57999 次迭代中的资源申请量为: 38.545164121939166
agent 13 在 57999 次迭代中的资源申请量为: 49.87892993968458
agent 14 在 57999 次迭代中的资源申请量为: 41.50244628948599
agent 15 在 57999 次迭代中的资源申请量为: 42.64082795933531
agent 16 在 57999 次迭代中的资源申请量为: 49.418927021637614
agent 17 在 57999 次迭代中的资源申请量为: 50.278380655626904
agent 18 在 57999 次迭代中的资源申请量为: 46.711864516741734
agent 19 在 57999 次迭代中的资源申请量为: 44.944698903213315
agent 20 在 57999 次迭代中的资源申请量为: 50.96604022350041
agent 21 在 57999 次迭代中的资源申请量为

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 56.3773885359945
agent 2 在 59999 次迭代中的资源申请量为: 56.20670471401895
agent 3 在 59999 次迭代中的资源申请量为: 60.86270576327959
agent 4 在 59999 次迭代中的资源申请量为: 38.05456524759511
agent 5 在 59999 次迭代中的资源申请量为: 57.75216512558106
agent 6 在 59999 次迭代中的资源申请量为: 55.53624349344336
agent 7 在 59999 次迭代中的资源申请量为: 65.85662660233905
agent 8 在 59999 次迭代中的资源申请量为: 46.50341640346253
agent 9 在 59999 次迭代中的资源申请量为: 65.85974500722061
agent 10 在 59999 次迭代中的资源申请量为: 54.400096430280875
agent 11 在 59999 次迭代中的资源申请量为: 65.67390625753639
agent 12 在 59999 次迭代中的资源申请量为: 38.545089748602464
agent 13 在 59999 次迭代中的资源申请量为: 49.88159271395787
agent 14 在 59999 次迭代中的资源申请量为: 41.50191258561869
agent 15 在 59999 次迭代中的资源申请量为: 42.64077943771322
agent 16 在 59999 次迭代中的资源申请量为: 49.418835153380385
agent 17 在 59999 次迭代中的资源申请量为: 50.27845361369593
agent 18 在 59999 次迭代中的资源申请量为: 46.7115047912937
agent 19 在 59999 次迭代中的资源申请量为: 44.944623484711926
agent 20 在 59999 次迭代中的资源申请量为: 50.966046448768594
agent 21 在 59999 次迭代中的资源申请量为: 7

In [12]:
np.savetxt('./data/01_IOS_q=4_agent_resource_quantity_max_attack.txt',agent_resource_quantity_robust2,fmt='%0.8f')#用户资源
np.savetxt('./data/01_IOS_q=4_average_cost_honest_max_attack.txt',average_cost_honest_robust2,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/01_IOS_q=4_average_resource_honest_max_attack.txt',average_resource_honest_robust2,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/01_IOS_q=4_dual_variable_max_attack.txt',dual_variable_robust2,fmt='%0.8f')#用户对偶变量

# DGD+IOS算法+大值高斯攻击1

In [13]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [14]:
#定义变量 
agent_resource_quantity_robust3 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust3=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust3=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust3 = 0.6
dual_variable_robust3 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust3= 0.06 #100个用户中拜占庭用户的比例 100*proportion_robust1=拜占庭用户数量
iteration_robust3=[]#绘图横坐标
A_robust3 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵 对角元素为0
neighbor_robust3_help=[]
q_robust3=[] #IOS鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量
W_robust3=np.loadtxt("100_Network_Topology_Adjacency_Matrix_B_0.0625(the value of bii=0.0625).txt") #双随机的网络拓扑矩阵W（对角线处非零）
a_robust3=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust3=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
random_attack3_robust3= np.loadtxt("./data/random_Gaussian_attack_value_-30_5.txt")
check_robust3=np.zeros(shape=(100,60000))
q_robust3=4 #所有诚实用户邻居中拜占庭用户数量的上界

In [15]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust3*100)):
    dual_variable_robust3[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust3*100)),100):
    dual_variable_robust3[i][0]=random_attack3_robust3[i-94][0]
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust3*100)):
    agent_resource_quantity_robust3[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust3=0
for i in range (0,100-int(proportion_robust3*100)):
    sum_robust3 = sum_robust3 + agent_resource_quantity_robust3[i][0]
average_resource_honest_robust3[0][0]=sum_robust3/(100-int(proportion_robust3*100))

#诚实用户平均成本初始化
sum_robust3=0
for i in range(0,100-int(proportion_robust3*100)):
    sum_robust3=sum_robust3+a_robust3[i]*(agent_resource_quantity_robust3[i][0]-b_robust3[i])*(agent_resource_quantity_robust3[i][0]-b_robust3[i])
average_cost_honest_robust3[0][0]=sum_robust3/(100-int(proportion_robust3*100))

iteration_robust3.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust3[i][j]!=0:
            neighbor_robust3_help.append(j)
neighbor_robust3=np.array(neighbor_robust3_help).reshape(100,15)

'''#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust3_help=0
    for j in range(0,15):
        if neighbor_robust3[i][j]==94 or neighbor_robust3[i][j]==95 or neighbor_robust3[i][j]==96 or neighbor_robust3[i][j]==97 or neighbor_robust3[i][j]==98 or neighbor_robust3[i][j]==99:
            q_robust3_help=q_robust3_help+1
    q_robust3.append(q_robust3_help) '''

'#计算诚实用户的拜占庭邻居用户数量\nfor i in range(0,100):\n    q_robust3_help=0\n    for j in range(0,15):\n        if neighbor_robust3[i][j]==94 or neighbor_robust3[i][j]==95 or neighbor_robust3[i][j]==96 or neighbor_robust3[i][j]==97 or neighbor_robust3[i][j]==98 or neighbor_robust3[i][j]==99:\n            q_robust3_help=q_robust3_help+1\n    q_robust3.append(q_robust3_help) '

In [16]:
#DGD+IOS算法 
def Robust_DGD_IOS_robust3(a_robust3,b_robust3,agent_resource_quantity_robust3,average_resource_honest_robust3,average_cost_honest_robust3,iteration_update_robust3,dual_variable_robust3,A_robust3,iteration_robust3,q_robust3):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+IOS+高斯拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust3[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust3[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust3[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust3[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust3)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust3.append(k)        
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust3=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust3*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust3.append(dual_variable_robust3[i][k-1]-iteration_update_robust3*((50/100)-(1/100)*agent_resource_quantity_robust3[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust3*100)),100):
            dual_variable_intermediate_robust3.append(dual_variable_robust3[i][k-1])
        #阶段二：IOS鲁棒聚合 每个用户聚合自身以及邻居的对偶值    
        for i in range (0,100-int(proportion_robust3*100)):
            u_neighbor_help_robust3=neighbor_robust3[i].tolist()#将用户i的邻居序号存在辅助变量u中
            u_neighbor_help_robust3.append(i)
            for q in range(0, int(q_robust3)):#剔除q个邻居的信息
                #计算当前的h-权重
                dual_variable_weight_average_robust3=0
                h_robust3=0
                for j in range(0,len(u_neighbor_help_robust3)):#计算当前循环中的h值
                    h_robust3=h_robust3+W_robust3[i][u_neighbor_help_robust3[j]]
                for j in range(0,len(u_neighbor_help_robust3)):#计算当前循环中的对偶权重平均值
                    dual_variable_weight_average_robust3=dual_variable_weight_average_robust3+(1/h_robust3)*(W_robust3[i][u_neighbor_help_robust3[j]]*dual_variable_intermediate_robust3[u_neighbor_help_robust3[j]])
                #剔除一个对偶值离权重平均对偶值最远的用户 保留自身的值
                u_neighbor_help_robust3.remove(i)
                neighbor_index_robust3=u_neighbor_help_robust3[0]
                max_distance_roubust3=abs(dual_variable_intermediate_robust3[u_neighbor_help_robust3[0]]-dual_variable_weight_average_robust3)
                for j in range(1,len(u_neighbor_help_robust3)):
                    if abs(dual_variable_intermediate_robust3[u_neighbor_help_robust3[j]]-dual_variable_weight_average_robust3)>=max_distance_roubust3:
                        max_distance_roubust3=abs(dual_variable_intermediate_robust3[u_neighbor_help_robust3[j]]-dual_variable_weight_average_robust3)
                        neighbor_index_robust3=u_neighbor_help_robust3[j]
                u_neighbor_help_robust3.remove(neighbor_index_robust3)
                u_neighbor_help_robust3.append(i)
            #计算剔除q个信息后的权重平均对偶值
            dual_variable_weight_average_robust3=0
            h_robust3=0
            for j in range(0,len(u_neighbor_help_robust3)):
                h_robust3=h_robust3+W_robust3[i][u_neighbor_help_robust3[j]]
            for j in range(0,len(u_neighbor_help_robust3)):
                dual_variable_weight_average_robust3=dual_variable_weight_average_robust3+(1/h_robust3)*(W_robust3[i][u_neighbor_help_robust3[j]]*dual_variable_intermediate_robust3[u_neighbor_help_robust3[j]])     
            #诚实用户基于IOS鲁棒聚合后的值进行自身的对偶变量的更新 基于权重平均值
            dual_variable_robust3[i][k]=dual_variable_weight_average_robust3
            
            '''#检测是否将拜占庭用户剔除
            for j in range(0,len(u_neighbor_help_robust3)):
                if u_neighbor_help_robust3[j]==94 or u_neighbor_help_robust3[j]==95 or u_neighbor_help_robust3[j]==96 or u_neighbor_help_robust3[j]==97 or u_neighbor_help_robust3[j]==98 or u_neighbor_help_robust3[j]==99:
                    check_robust3[i][k]=1 '''            
            
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust3*100),100):
            dual_variable_robust3[i][k]=random_attack3_robust3[i-94][k]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust3*100)):
            agent_resource_quantity_robust3[i][k]=(dual_variable_robust3[i][k]-2*a_robust3[i]*b_robust3[i])/(-2*a_robust3[i])
            #原变量的投影操作
            if agent_resource_quantity_robust3[i][k] < 0:
                agent_resource_quantity_robust3[i][k]=0
            elif agent_resource_quantity_robust3[i][k] > 100: 
                agent_resource_quantity_robust3[i][k]=100  
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust3*100),100):
            agent_resource_quantity_robust3[i][k]=agent_resource_quantity_robust3[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust3=0
        for i in range (0,100-int(proportion_robust3*100)):
            sum1_robust3 = sum1_robust3 + agent_resource_quantity_robust3[i][k]
        average_resource_honest_robust3[0][k]=sum1_robust3/(100-int(proportion_robust3*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust3=0
        for i in range(0,100-int(proportion_robust3*100)):
            sum2_robust3=sum2_robust3+a_robust3[i]*(agent_resource_quantity_robust3[i][k]-b_robust3[i])*(agent_resource_quantity_robust3[i][k]-b_robust3[i])
        average_cost_honest_robust3[0][k]=sum2_robust3/(100-int(proportion_robust3*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust3[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust3[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust3[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust3[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust3)
            print("****************************************************************************")
            '''print("聚合权重为:",h_robust3)
            print("****************************************************************************")'''
        
        iteration_update_robust3=1/(pow(k+1,0.1)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [17]:
Robust_DGD_IOS_robust3(a_robust3,b_robust3,agent_resource_quantity_robust3,average_resource_honest_robust3,average_cost_honest_robust3,iteration_update_robust3,dual_variable_robust3,A_robust3,iteration_robust3,q_robust3)

****************************DGD算法+IOS+高斯拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 56.5449312570376
agent 2 在 3999 次迭代中的资源申请量为: 56.3695327081338
agent 3 在 3999 次迭代中的资源申请量为: 61.0522699179049
agent 4 在 3999 次迭代中的资源申请量为: 38.16776446621911
agent 5 在 3999 次迭代中的资源申请量为: 57.923357254403356
agent 6 在 3999 次迭代中的资源申请量为: 55.706081492753775
agent 7 在 3999 次迭代中的资源申请量为: 66.0624067478197
agent 8 在 3999 次迭代中的资源申请量为: 46.649231221739335
agent 9 在 3999 次迭代中的资源申请量为: 66.05213440680792
agent 10 在 3999 次迭代中的资源申请量为: 54.55773046761518
agent 11 在 3999 次迭代中的资源申请量为: 65.87727746456389
agent 12 在 3999 次迭代中的资源申请量为: 38.66434894112626
agent 13 在 3999 次迭代中的资源申请量为: 50.03522068728005
agent 14 在 3999 次迭代中的资源申请量为: 41.62597225819531
agent 15 在 3999 次迭代中的资源申请量为: 42.77177360999312
agent 16 在 3999 次迭代中的资源申请量为: 49.570350897601585
agent 17 在 3999 次迭代中的资源申请量为: 50.42657108361426
agent 18 在 3999 次迭代中的资源申请量为: 46.858342724378005
agent 19 在 3999 次迭代中的资源申请量为: 45.0879790359273
agent 20 在 3999 次迭代中的资源申请量为: 51.11639565386603
agent 21 在 3999 次迭代中的资源申请量为: 70.36456217091585
agent 22

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 56.39118479451008
agent 2 在 5999 次迭代中的资源申请量为: 56.21790151593966
agent 3 在 5999 次迭代中的资源申请量为: 60.88264833266711
agent 4 在 5999 次迭代中的资源申请量为: 38.06330317555701
agent 5 在 5999 次迭代中的资源申请量为: 57.76641316776792
agent 6 在 5999 次迭代中的资源申请量为: 55.554755693122786
agent 7 在 5999 次迭代中的资源申请量为: 65.87817110211219
agent 8 在 5999 次迭代中的资源申请量为: 46.52097251583744
agent 9 在 5999 次迭代中的资源申请量为: 65.87105943090442
agent 10 在 5999 次迭代中的资源申请量为: 54.40903235026756
agent 11 在 5999 次迭代中的资源申请量为: 65.68364291467972
agent 12 在 5999 次迭代中的资源申请量为: 38.55889994657645
agent 13 在 5999 次迭代中的资源申请量为: 49.89903812064728
agent 14 在 5999 次迭代中的资源申请量为: 41.51348236608885
agent 15 在 5999 次迭代中的资源申请量为: 42.655832974584236
agent 16 在 5999 次迭代中的资源申请量为: 49.43539259103372
agent 17 在 5999 次迭代中的资源申请量为: 50.29506330357174
agent 18 在 5999 次迭代中的资源申请量为: 46.729756326656116
agent 19 在 5999 次迭代中的资源申请量为: 44.96255508521241
agent 20 在 5999 次迭代中的资源申请量为: 50.97909864090019
agent 21 在 5999 次迭代中的资源申请量为: 70.17154061669238
agent

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 56.37870834601807
agent 2 在 7999 次迭代中的资源申请量为: 56.205289507780805
agent 3 在 7999 次迭代中的资源申请量为: 60.86796150802477
agent 4 在 7999 次迭代中的资源申请量为: 38.05489058123868
agent 5 在 7999 次迭代中的资源申请量为: 57.75352285165003
agent 6 在 7999 次迭代中的资源申请量为: 55.54084379829204
agent 7 在 7999 次迭代中的资源申请量为: 65.862045306702
agent 8 在 7999 次迭代中的资源申请量为: 46.50982863422326
agent 9 在 7999 次迭代中的资源申请量为: 65.86156286562056
agent 10 在 7999 次迭代中的资源申请量为: 54.39779725187115
agent 11 在 7999 次迭代中的资源申请量为: 65.67578072161109
agent 12 在 7999 次迭代中的资源申请量为: 38.549311425492604
agent 13 在 7999 次迭代中的资源申请量为: 49.886723683245734
agent 14 在 7999 次迭代中的资源申请量为: 41.503659739613504
agent 15 在 7999 次迭代中的资源申请量为: 42.64556594218502
agent 16 在 7999 次迭代中的资源申请量为: 49.423500271870836
agent 17 在 7999 次迭代中的资源申请量为: 50.279363803287694
agent 18 在 7999 次迭代中的资源申请量为: 46.718237133403186
agent 19 在 7999 次迭代中的资源申请量为: 44.950940700576744
agent 20 在 7999 次迭代中的资源申请量为: 50.96738803112814
agent 21 在 7999 次迭代中的资源申请量为: 70.15607677580172
ag

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 56.37569270548906
agent 2 在 9999 次迭代中的资源申请量为: 56.20295830410901
agent 3 在 9999 次迭代中的资源申请量为: 60.864854019562465
agent 4 在 9999 次迭代中的资源申请量为: 38.0536186591189
agent 5 在 9999 次迭代中的资源申请量为: 57.750256974223014
agent 6 在 9999 次迭代中的资源申请量为: 55.538127002903785
agent 7 在 9999 次迭代中的资源申请量为: 65.85819542181572
agent 8 在 9999 次迭代中的资源申请量为: 46.507269957427994
agent 9 在 9999 次迭代中的资源申请量为: 65.85306868479047
agent 10 在 9999 次迭代中的资源申请量为: 54.396017346821296
agent 11 在 9999 次迭代中的资源申请量为: 65.67325268159624
agent 12 在 9999 次迭代中的资源申请量为: 38.547197152442834
agent 13 在 9999 次迭代中的资源申请量为: 49.88038403863745
agent 14 在 9999 次迭代中的资源申请量为: 41.50236622022894
agent 15 在 9999 次迭代中的资源申请量为: 42.64293018221724
agent 16 在 9999 次迭代中的资源申请量为: 49.42059296800191
agent 17 在 9999 次迭代中的资源申请量为: 50.277048541170366
agent 18 在 9999 次迭代中的资源申请量为: 46.715332802854455
agent 19 在 9999 次迭代中的资源申请量为: 44.94817819349893
agent 20 在 9999 次迭代中的资源申请量为: 50.96504218596132
agent 21 在 9999 次迭代中的资源申请量为: 70.15270039912266
a

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 56.37744281126447
agent 2 在 51999 次迭代中的资源申请量为: 56.20641758049134
agent 3 在 51999 次迭代中的资源申请量为: 60.86327186552412
agent 4 在 51999 次迭代中的资源申请量为: 38.05472820731289
agent 5 在 51999 次迭代中的资源申请量为: 57.75166756106454
agent 6 在 51999 次迭代中的资源申请量为: 55.536421066499216
agent 7 在 51999 次迭代中的资源申请量为: 65.85676306020686
agent 8 在 51999 次迭代中的资源申请量为: 46.504008651745494
agent 9 在 51999 次迭代中的资源申请量为: 65.85488891329554
agent 10 在 51999 次迭代中的资源申请量为: 54.39989623256886
agent 11 在 51999 次迭代中的资源申请量为: 65.67408014699963
agent 12 在 51999 次迭代中的资源申请量为: 38.54527869552301
agent 13 在 51999 次迭代中的资源申请量为: 49.87896870046594
agent 14 在 51999 次迭代中的资源申请量为: 41.50245587661907
agent 15 在 51999 次迭代中的资源申请量为: 42.64093669473941
agent 16 在 51999 次迭代中的资源申请量为: 49.41898143575325
agent 17 在 51999 次迭代中的资源申请量为: 50.278213313841306
agent 18 在 51999 次迭代中的资源申请量为: 46.71197885202628
agent 19 在 51999 次迭代中的资源申请量为: 44.9448673611821
agent 20 在 51999 次迭代中的资源申请量为: 50.96606344174816
agent 21 在 51999 次迭代中的资源申请量为: 70

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 56.377342066003976
agent 2 在 53999 次迭代中的资源申请量为: 56.206522036512645
agent 3 在 53999 次迭代中的资源申请量为: 60.86286818127115
agent 4 在 53999 次迭代中的资源申请量为: 38.05433472323231
agent 5 在 53999 次迭代中的资源申请量为: 57.75172925460147
agent 6 在 53999 次迭代中的资源申请量为: 55.53652853245121
agent 7 在 53999 次迭代中的资源申请量为: 65.85682338728749
agent 8 在 53999 次迭代中的资源申请量为: 46.50335275674147
agent 9 在 53999 次迭代中的资源申请量为: 65.85451144533448
agent 10 在 53999 次迭代中的资源申请量为: 54.3993349600315
agent 11 在 53999 次迭代中的资源申请量为: 65.67354681038135
agent 12 在 53999 次迭代中的资源申请量为: 38.54524046255062
agent 13 在 53999 次迭代中的资源申请量为: 49.88176079156025
agent 14 在 53999 次迭代中的资源申请量为: 41.50195731175776
agent 15 在 53999 次迭代中的资源申请量为: 42.640954204752774
agent 16 在 53999 次迭代中的资源申请量为: 49.41900103167398
agent 17 在 53999 次迭代中的资源申请量为: 50.27840920034348
agent 18 在 53999 次迭代中的资源申请量为: 46.71179551113095
agent 19 在 53999 次迭代中的资源申请量为: 44.944910851578456
agent 20 在 53999 次迭代中的资源申请量为: 50.966038516021904
agent 21 在 53999 次迭代中的资源申请量为: 

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 56.37774563133717
agent 2 在 55999 次迭代中的资源申请量为: 56.20645476528424
agent 3 在 55999 次迭代中的资源申请量为: 60.86286842876223
agent 4 在 55999 次迭代中的资源申请量为: 38.05428667811361
agent 5 在 55999 次迭代中的资源申请量为: 57.752065735524354
agent 6 在 55999 次迭代中的资源申请量为: 55.53596017383931
agent 7 在 55999 次迭代中的资源申请量为: 65.85675176969875
agent 8 在 55999 次迭代中的资源申请量为: 46.503554368695895
agent 9 在 55999 次迭代中的资源申请量为: 65.85927017917089
agent 10 在 55999 次迭代中的资源申请量为: 54.39960831604462
agent 11 在 55999 次迭代中的资源申请量为: 65.67332950898533
agent 12 在 55999 次迭代中的资源申请量为: 38.54502537102551
agent 13 在 55999 次迭代中的资源申请量为: 49.88172446473919
agent 14 在 55999 次迭代中的资源申请量为: 41.50174247814269
agent 15 在 55999 次迭代中的资源申请量为: 42.64093133925856
agent 16 在 55999 次迭代中的资源申请量为: 49.41895578327388
agent 17 在 55999 次迭代中的资源申请量为: 50.27813457658412
agent 18 在 55999 次迭代中的资源申请量为: 46.71177986217761
agent 19 在 55999 次迭代中的资源申请量为: 44.94460423570962
agent 20 在 55999 次迭代中的资源申请量为: 50.9659578920415
agent 21 在 55999 次迭代中的资源申请量为: 70.

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 56.37756032785262
agent 2 在 57999 次迭代中的资源申请量为: 56.20663858303221
agent 3 在 57999 次迭代中的资源申请量为: 60.86318851864434
agent 4 在 57999 次迭代中的资源申请量为: 38.05480318830534
agent 5 在 57999 次迭代中的资源申请量为: 57.75176475392852
agent 6 在 57999 次迭代中的资源申请量为: 55.5363288669035
agent 7 在 57999 次迭代中的资源申请量为: 65.85668989250667
agent 8 在 57999 次迭代中的资源申请量为: 46.50382039829603
agent 9 在 57999 次迭代中的资源申请量为: 65.85501274644452
agent 10 在 57999 次迭代中的资源申请量为: 54.40014239409317
agent 11 在 57999 次迭代中的资源申请量为: 65.67414406860986
agent 12 在 57999 次迭代中的资源申请量为: 38.54517021916743
agent 13 在 57999 次迭代中的资源申请量为: 49.878893261523196
agent 14 在 57999 次迭代中的资源申请量为: 41.502469627116895
agent 15 在 57999 次迭代中的资源申请量为: 42.6408244571847
agent 16 在 57999 次迭代中的资源申请量为: 49.41889374731726
agent 17 在 57999 次迭代中的资源申请量为: 50.27829399988868
agent 18 在 57999 次迭代中的资源申请量为: 46.71178503813926
agent 19 在 57999 次迭代中的资源申请量为: 44.944675958334855
agent 20 在 57999 次迭代中的资源申请量为: 50.96613554966954
agent 21 在 57999 次迭代中的资源申请量为: 70.

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 56.37745652853148
agent 2 在 59999 次迭代中的资源申请量为: 56.2067342258713
agent 3 在 59999 次迭代中的资源申请量为: 60.86279197805854
agent 4 在 59999 次迭代中的资源申请量为: 38.054411197229676
agent 5 在 59999 次迭代中的资源申请量为: 57.75182240615063
agent 6 在 59999 次迭代中的资源申请量为: 55.53643843981252
agent 7 在 59999 次迭代中的资源申请量为: 65.8567521527079
agent 8 在 59999 次迭代中的资源申请量为: 46.50317794877185
agent 9 在 59999 次迭代中的资源申请量为: 65.85463489003236
agent 10 在 59999 次迭代中的资源申请量为: 54.39957838256303
agent 11 在 59999 次迭代中的资源申请量为: 65.67361407905207
agent 12 在 59999 次迭代中的资源申请量为: 38.54513618397953
agent 13 在 59999 次迭代中的资源申请量为: 49.881658731146715
agent 14 在 59999 次迭代中的资源申请量为: 41.50197580159476
agent 15 在 59999 次迭代中的资源申请量为: 42.64084571314753
agent 16 在 59999 次迭代中的资源申请量为: 49.41891620850845
agent 17 在 59999 次迭代中的资源申请量为: 50.278485020865
agent 18 在 59999 次迭代中的资源申请量为: 46.71161039654875
agent 19 在 59999 次迭代中的资源申请量为: 44.94472569048964
agent 20 在 59999 次迭代中的资源申请量为: 50.966108367121706
agent 21 在 59999 次迭代中的资源申请量为: 70.15

In [18]:
np.savetxt('./data/01_IOS_q=4_agent_resource_quantity_Gaussian_attack_large.txt',agent_resource_quantity_robust3,fmt='%0.8f')#用户资源
np.savetxt('./data/01_IOS_q=4_average_cost_honest_Gaussian_attack_large.txt',average_cost_honest_robust3,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/01_IOS_q=4_average_resource_honest_Gaussian_attack_large.txt',average_resource_honest_robust3,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/01_IOS_q=4_dual_variable_Gaussian_attack_large.txt',dual_variable_robust3,fmt='%0.8f')#用户对偶变量

# DGD+IOS算法+小值高斯攻击2

In [19]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [20]:
#定义变量 
agent_resource_quantity_robust4 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust4=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust4=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust4 = 0.6
dual_variable_robust4 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust4= 0.06 #100个用户中拜占庭用户的比例 100*proportion_robust1=拜占庭用户数量
iteration_robust4=[]#绘图横坐标
A_robust4 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵 对角元素为0
neighbor_robust4_help=[]
q_robust4=[] #IOS鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量
W_robust4=np.loadtxt("100_Network_Topology_Adjacency_Matrix_B_0.0625(the value of bii=0.0625).txt") #双随机的网络拓扑矩阵W（对角线处非零）
a_robust4=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust4=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
random_attack3_robust4= np.loadtxt("./data/random_Gaussian_attack_value_-300_40.txt")
check_robust4=np.zeros(shape=(100,60000))
q_robust4=4 #所有诚实用户邻居中拜占庭用户数量的上界

In [21]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust4*100)):
    dual_variable_robust4[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust4*100)),100):
    dual_variable_robust4[i][0]=random_attack3_robust4[i-94][0]
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust4*100)):
    agent_resource_quantity_robust4[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust4=0
for i in range (0,100-int(proportion_robust4*100)):
    sum_robust4 = sum_robust4 + agent_resource_quantity_robust4[i][0]
average_resource_honest_robust4[0][0]=sum_robust4/(100-int(proportion_robust4*100))

#诚实用户平均成本初始化
sum_robust4=0
for i in range(0,100-int(proportion_robust4*100)):
    sum_robust4=sum_robust4+a_robust4[i]*(agent_resource_quantity_robust4[i][0]-b_robust4[i])*(agent_resource_quantity_robust4[i][0]-b_robust4[i])
average_cost_honest_robust4[0][0]=sum_robust4/(100-int(proportion_robust4*100))

iteration_robust4.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust4[i][j]!=0:
            neighbor_robust4_help.append(j)
neighbor_robust4=np.array(neighbor_robust4_help).reshape(100,15)

'''#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust4_help=0
    for j in range(0,15):
        if neighbor_robust4[i][j]==94 or neighbor_robust4[i][j]==95 or neighbor_robust4[i][j]==96 or neighbor_robust4[i][j]==97 or neighbor_robust4[i][j]==98 or neighbor_robust4[i][j]==99:
            q_robust4_help=q_robust4_help+1
    q_robust4.append(q_robust4_help)'''

'#计算诚实用户的拜占庭邻居用户数量\nfor i in range(0,100):\n    q_robust4_help=0\n    for j in range(0,15):\n        if neighbor_robust4[i][j]==94 or neighbor_robust4[i][j]==95 or neighbor_robust4[i][j]==96 or neighbor_robust4[i][j]==97 or neighbor_robust4[i][j]==98 or neighbor_robust4[i][j]==99:\n            q_robust4_help=q_robust4_help+1\n    q_robust4.append(q_robust4_help)'

In [22]:
#DGD+IOS算法 
def Robust_DGD_IOS_robust4(a_robust4,b_robust4,agent_resource_quantity_robust4,average_resource_honest_robust4,average_cost_honest_robust4,iteration_update_robust4,dual_variable_robust4,A_robust4,iteration_robust4,q_robust4):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+IOS+高斯拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust4[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust4[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust4[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust4[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust4)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust4.append(k)        
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust4=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust4*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust4.append(dual_variable_robust4[i][k-1]-iteration_update_robust4*((50/100)-(1/100)*agent_resource_quantity_robust4[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust4*100)),100):
            dual_variable_intermediate_robust4.append(dual_variable_robust4[i][k-1])
        #阶段二：IOS鲁棒聚合 每个用户聚合自身以及邻居的对偶值    
        for i in range (0,100-int(proportion_robust4*100)):
            u_neighbor_help_robust4=neighbor_robust4[i].tolist()#将用户i的邻居序号存在辅助变量u中
            u_neighbor_help_robust4.append(i)
            for q in range(0, int(q_robust4)):#剔除q个邻居的信息
                #计算当前的h-权重
                dual_variable_weight_average_robust4=0
                h_robust4=0
                for j in range(0,len(u_neighbor_help_robust4)):#计算当前循环中的h值
                    h_robust4=h_robust4+W_robust4[i][u_neighbor_help_robust4[j]]
                for j in range(0,len(u_neighbor_help_robust4)):#计算当前循环中的对偶权重平均值
                    dual_variable_weight_average_robust4=dual_variable_weight_average_robust4+(1/h_robust4)*(W_robust4[i][u_neighbor_help_robust4[j]]*dual_variable_intermediate_robust4[u_neighbor_help_robust4[j]])
                #剔除一个对偶值离权重平均对偶值最远的用户 保留自身的值
                u_neighbor_help_robust4.remove(i)
                neighbor_index_robust4=u_neighbor_help_robust4[0]
                max_distance_roubust4=abs(dual_variable_intermediate_robust4[u_neighbor_help_robust4[0]]-dual_variable_weight_average_robust4)
                for j in range(1,len(u_neighbor_help_robust4)):
                    if abs(dual_variable_intermediate_robust4[u_neighbor_help_robust4[j]]-dual_variable_weight_average_robust4)>=max_distance_roubust4:
                        max_distance_roubust4=abs(dual_variable_intermediate_robust4[u_neighbor_help_robust4[j]]-dual_variable_weight_average_robust4)
                        neighbor_index_robust4=u_neighbor_help_robust4[j]
                u_neighbor_help_robust4.remove(neighbor_index_robust4)
                u_neighbor_help_robust4.append(i)
            #计算剔除q个信息后的权重平均对偶值
            dual_variable_weight_average_robust4=0
            h_robust4=0
            for j in range(0,len(u_neighbor_help_robust4)):
                h_robust4=h_robust4+W_robust4[i][u_neighbor_help_robust4[j]]
            for j in range(0,len(u_neighbor_help_robust4)):
                dual_variable_weight_average_robust4=dual_variable_weight_average_robust4+(1/h_robust4)*(W_robust4[i][u_neighbor_help_robust4[j]]*dual_variable_intermediate_robust4[u_neighbor_help_robust4[j]])     
            #诚实用户基于IOS鲁棒聚合后的值进行自身的对偶变量的更新 基于权重平均值
            dual_variable_robust4[i][k]=dual_variable_weight_average_robust4
            
            '''#检测是否将拜占庭用户剔除
            for j in range(0,len(u_neighbor_help_robust4)):
                if u_neighbor_help_robust4[j]==94 or u_neighbor_help_robust4[j]==95 or u_neighbor_help_robust4[j]==96 or u_neighbor_help_robust4[j]==97 or u_neighbor_help_robust4[j]==98 or u_neighbor_help_robust4[j]==99:
                    check_robust4[i][k]=1 '''            
            
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust4*100),100):
            dual_variable_robust4[i][k]=random_attack3_robust4[i-94][k]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust4*100)):
            agent_resource_quantity_robust4[i][k]=(dual_variable_robust4[i][k]-2*a_robust4[i]*b_robust4[i])/(-2*a_robust4[i])
            #原变量的投影操作
            if agent_resource_quantity_robust4[i][k] < 0:
                agent_resource_quantity_robust4[i][k]=0
            elif agent_resource_quantity_robust4[i][k] > 100: 
                agent_resource_quantity_robust4[i][k]=100  
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust4*100),100):
            agent_resource_quantity_robust4[i][k]=agent_resource_quantity_robust4[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust4=0
        for i in range (0,100-int(proportion_robust4*100)):
            sum1_robust4 = sum1_robust4 + agent_resource_quantity_robust4[i][k]
        average_resource_honest_robust4[0][k]=sum1_robust4/(100-int(proportion_robust4*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust4=0
        for i in range(0,100-int(proportion_robust4*100)):
            sum2_robust4=sum2_robust4+a_robust4[i]*(agent_resource_quantity_robust4[i][k]-b_robust4[i])*(agent_resource_quantity_robust4[i][k]-b_robust4[i])
        average_cost_honest_robust4[0][k]=sum2_robust4/(100-int(proportion_robust4*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust4[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust4[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust4[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust4[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust4)
            print("****************************************************************************")
            '''print("聚合权重为:",h_robust3)
            print("****************************************************************************")'''
        
        iteration_update_robust4=1/(pow(k+1,0.1)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [23]:
Robust_DGD_IOS_robust4(a_robust4,b_robust4,agent_resource_quantity_robust4,average_resource_honest_robust4,average_cost_honest_robust4,iteration_update_robust4,dual_variable_robust4,A_robust4,iteration_robust4,q_robust4)

****************************DGD算法+IOS+高斯拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 56.57291376360089
agent 2 在 3999 次迭代中的资源申请量为: 56.39722173349602
agent 3 在 3999 次迭代中的资源申请量为: 61.08302575264673
agent 4 在 3999 次迭代中的资源申请量为: 38.1867233654013
agent 5 在 3999 次迭代中的资源申请量为: 57.9518291541347
agent 6 在 3999 次迭代中的资源申请量为: 55.73321685123774
agent 7 在 3999 次迭代中的资源申请量为: 66.09588984297694
agent 8 在 3999 次迭代中的资源申请量为: 46.671782714045634
agent 9 在 3999 次迭代中的资源申请量为: 66.08514772481054
agent 10 在 3999 次迭代中的资源申请量为: 54.584260025154705
agent 11 在 3999 次迭代中的资源申请量为: 65.90964109257676
agent 12 在 3999 次迭代中的资源申请量为: 38.683431365494826
agent 13 在 3999 次迭代中的资源申请量为: 50.06065594489135
agent 14 在 3999 次迭代中的资源申请量为: 41.64633511029743
agent 15 在 3999 次迭代中的资源申请量为: 42.79281860110122
agent 16 在 3999 次迭代中的资源申请量为: 49.594885739619585
agent 17 在 3999 次迭代中的资源申请量为: 50.45120133075163
agent 18 在 3999 次迭代中的资源申请量为: 46.881645983260334
agent 19 在 3999 次迭代中的资源申请量为: 45.110719495772784
agent 20 在 3999 次迭代中的资源申请量为: 51.14133719819595
agent 21 在 3999 次迭代中的资源申请量为: 70.3994138834136
agent

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 56.39244423352228
agent 2 在 5999 次迭代中的资源申请量为: 56.219246374582
agent 3 在 5999 次迭代中的资源申请量为: 60.884137963983456
agent 4 在 5999 次迭代中的资源申请量为: 38.06483723432002
agent 5 在 5999 次迭代中的资源申请量为: 57.76824663821338
agent 6 在 5999 次迭代中的资源申请量为: 55.55592443461221
agent 7 在 5999 次迭代中的资源申请量为: 65.87969211990644
agent 8 在 5999 次迭代中的资源申请量为: 46.522445091576465
agent 9 在 5999 次迭代中的资源申请量为: 65.87914195356397
agent 10 在 5999 次迭代中的资源申请量为: 54.41157829843846
agent 11 在 5999 次迭代中的资源申请量为: 65.69384078491187
agent 12 在 5999 次迭代中的资源申请量为: 38.55980546811647
agent 13 在 5999 次迭代中的资源申请量为: 49.90024601637019
agent 14 在 5999 次迭代中的资源申请量为: 41.51443569293387
agent 15 在 5999 次迭代中的资源申请量为: 42.656818447921076
agent 16 在 5999 次迭代中的资源申请量为: 49.43650998553947
agent 17 在 5999 次迭代中的资源申请量为: 50.292357203575435
agent 18 在 5999 次迭代中的资源申请量为: 46.7307999272165
agent 19 在 5999 次迭代中的资源申请量为: 44.96358998887069
agent 20 在 5999 次迭代中的资源申请量为: 50.98029608895721
agent 21 在 5999 次迭代中的资源申请量为: 70.17392755482683
agent 2

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 56.37872558404526
agent 2 在 7999 次迭代中的资源申请量为: 56.20560157245145
agent 3 在 7999 次迭代中的资源申请量为: 60.868238079293256
agent 4 在 7999 次迭代中的资源申请量为: 38.05511942210575
agent 5 在 7999 次迭代中的资源申请量为: 57.75332302682588
agent 6 在 7999 次迭代中的资源申请量为: 55.54152168828745
agent 7 在 7999 次迭代中的资源申请量为: 65.86234771616884
agent 8 在 7999 次迭代中的资源申请量为: 46.509774522639724
agent 9 在 7999 次迭代中的资源申请量为: 65.85598592813955
agent 10 在 7999 次迭代中的资源申请量为: 54.39792596115451
agent 11 在 7999 次迭代中的资源申请量为: 65.67647587295149
agent 12 在 7999 次迭代中的资源申请量为: 38.549456166674304
agent 13 在 7999 次迭代中的资源申请量为: 49.88684161155929
agent 14 在 7999 次迭代中的资源申请量为: 41.50389608191523
agent 15 在 7999 次迭代中的资源申请量为: 42.64569853493217
agent 16 在 7999 次迭代中的资源申请量为: 49.42367418645401
agent 17 在 7999 次迭代中的资源申请量为: 50.27945350516737
agent 18 在 7999 次迭代中的资源申请量为: 46.718339998030046
agent 19 在 7999 次迭代中的资源申请量为: 44.951130199398385
agent 20 在 7999 次迭代中的资源申请量为: 50.967698741811894
agent 21 在 7999 次迭代中的资源申请量为: 70.15575997016403
ag

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 56.37587053242325
agent 2 在 9999 次迭代中的资源申请量为: 56.202796702780965
agent 3 在 9999 次迭代中的资源申请量为: 60.8648294640941
agent 4 在 9999 次迭代中的资源申请量为: 38.05329459917122
agent 5 在 9999 次迭代中的资源申请量为: 57.75017679188492
agent 6 在 9999 次迭代中的资源申请量为: 55.5378218498249
agent 7 在 9999 次迭代中的资源申请量为: 65.85820923020168
agent 8 在 9999 次迭代中的资源申请量为: 46.5068345035661
agent 9 在 9999 次迭代中的资源申请量为: 65.85250802838203
agent 10 在 9999 次迭代中的资源申请量为: 54.39503545975119
agent 11 在 9999 次迭代中的资源申请量为: 65.67192011079423
agent 12 在 9999 次迭代中的资源申请量为: 38.547194103362536
agent 13 在 9999 次迭代中的资源申请量为: 49.880433334976
agent 14 在 9999 次迭代中的资源申请量为: 41.5023429138488
agent 15 在 9999 次迭代中的资源申请量为: 42.64293900714455
agent 16 在 9999 次迭代中的资源申请量为: 49.42063803384619
agent 17 在 9999 次迭代中的资源申请量为: 50.27715731976509
agent 18 在 9999 次迭代中的资源申请量为: 46.715432644534246
agent 19 在 9999 次迭代中的资源申请量为: 44.948210568911435
agent 20 在 9999 次迭代中的资源申请量为: 50.964934129401506
agent 21 在 9999 次迭代中的资源申请量为: 70.15261823916069
agent 22 

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 56.37758839171811
agent 2 在 51999 次迭代中的资源申请量为: 56.206275261234865
agent 3 在 51999 次迭代中的资源申请量为: 60.86324527198883
agent 4 在 51999 次迭代中的资源申请量为: 38.05444965605938
agent 5 在 51999 次迭代中的资源申请量为: 57.75159408523659
agent 6 在 51999 次迭代中的资源申请量为: 55.53615705154922
agent 7 在 51999 次迭代中的资源申请量为: 65.85676850598307
agent 8 在 51999 次迭代中的资源申请量为: 46.50363487086908
agent 9 在 51999 次迭代中的资源申请量为: 65.85440683132869
agent 10 在 51999 次迭代中的资源申请量为: 54.399058004599716
agent 11 在 51999 次迭代中的资源申请量为: 65.67294324600513
agent 12 在 51999 次迭代中的资源申请量为: 38.54527252937365
agent 13 在 51999 次迭代中的资源申请量为: 49.87900577695267
agent 14 在 51999 次迭代中的资源申请量为: 41.502432281344014
agent 15 在 51999 次迭代中的资源申请量为: 42.64094023340903
agent 16 在 51999 次迭代中的资源申请量为: 49.41901507201586
agent 17 在 51999 次迭代中的资源申请量为: 50.27830091877743
agent 18 在 51999 次迭代中的资源申请量为: 46.71205919888747
agent 19 在 51999 次迭代中的资源申请量为: 44.94489055599918
agent 20 在 51999 次迭代中的资源申请量为: 50.9659670691321
agent 21 在 51999 次迭代中的资源申请量为: 70

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 56.37727335814798
agent 2 在 53999 次迭代中的资源申请量为: 56.20649222079688
agent 3 在 53999 次迭代中的资源申请量为: 60.862781058031615
agent 4 在 53999 次迭代中的资源申请量为: 38.05449040056214
agent 5 在 53999 次迭代中的资源申请量为: 57.75207560571343
agent 6 在 53999 次迭代中的资源申请量为: 55.53633152944671
agent 7 在 53999 次迭代中的资源申请量为: 65.85669651372427
agent 8 在 53999 次迭代中的资源申请量为: 46.503593732041075
agent 9 在 53999 次迭代中的资源申请量为: 65.85967555043514
agent 10 在 53999 次迭代中的资源申请量为: 54.39985848116011
agent 11 在 53999 次迭代中的资源申请量为: 65.67384208200173
agent 12 在 53999 次迭代中的资源申请量为: 38.54519353793994
agent 13 在 53999 次迭代中的资源申请量为: 49.88169407878632
agent 14 在 53999 次迭代中的资源申请量为: 41.50189343195017
agent 15 在 53999 次迭代中的资源申请量为: 42.64088723060619
agent 16 在 53999 次迭代中的资源申请量为: 49.41891912107662
agent 17 在 53999 次迭代中的资源申请量为: 50.278377463316865
agent 18 在 53999 次迭代中的资源申请量为: 46.71168879067632
agent 19 在 53999 次迭代中的资源申请量为: 44.94480756694356
agent 20 在 53999 次迭代中的资源申请量为: 50.96597594715283
agent 21 在 53999 次迭代中的资源申请量为: 7

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 56.377671067410056
agent 2 在 55999 次迭代中的资源申请量为: 56.20662374710079
agent 3 在 55999 次迭代中的资源申请量为: 60.86299885203942
agent 4 在 55999 次迭代中的资源申请量为: 38.0544150454895
agent 5 在 55999 次迭代中的资源申请量为: 57.751810644101894
agent 6 在 55999 次迭代中的资源申请量为: 55.53643132718092
agent 7 在 55999 次迭代中的资源申请量为: 65.85689468788337
agent 8 在 55999 次迭代中的资源申请量为: 46.50343678754689
agent 9 在 55999 次迭代中的资源申请量为: 65.85457268260036
agent 10 在 55999 次迭代中的资源申请量为: 54.39962845549212
agent 11 在 55999 次迭代中的资源申请量为: 65.67379526831851
agent 12 在 55999 次迭代中的资源申请量为: 38.5450840473914
agent 13 在 55999 次迭代中的资源申请量为: 49.88174118147473
agent 14 在 55999 次迭代中的资源申请量为: 41.50187261780079
agent 15 在 55999 次迭代中的资源申请量为: 42.64097392111692
agent 16 在 55999 次迭代中的资源申请量为: 49.41902136060781
agent 17 在 55999 次迭代中的资源申请量为: 50.27813018437617
agent 18 在 55999 次迭代中的资源申请量为: 46.71179100989178
agent 19 在 55999 次迭代中的资源申请量为: 44.94468840019774
agent 20 在 55999 次迭代中的资源申请量为: 50.96613445704986
agent 21 在 55999 次迭代中的资源申请量为: 70.1

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 56.377704332376
agent 2 在 57999 次迭代中的资源申请量为: 56.206497808189305
agent 3 在 57999 次迭代中的资源申请量为: 60.86316221641174
agent 4 在 57999 次迭代中的资源申请量为: 38.054527652552146
agent 5 在 57999 次迭代中的资源申请量为: 57.751692072278104
agent 6 在 57999 次迭代中的资源申请量为: 55.53606771647086
agent 7 在 57999 次迭代中的资源申请量为: 65.85669528348942
agent 8 在 57999 次迭代中的资源申请量为: 46.50345066725779
agent 9 在 57999 次迭代中的资源申请量为: 65.8545358763489
agent 10 在 57999 次迭代中的资源申请量为: 54.399313241773335
agent 11 在 57999 次迭代中的资源申请量为: 65.67301948408095
agent 12 在 57999 次迭代中的资源申请量为: 38.545164121939166
agent 13 在 57999 次迭代中的资源申请量为: 49.87892993968458
agent 14 在 57999 次迭代中的资源申请量为: 41.50244628948599
agent 15 在 57999 次迭代中的资源申请量为: 42.64082795933531
agent 16 在 57999 次迭代中的资源申请量为: 49.418927021637614
agent 17 在 57999 次迭代中的资源申请量为: 50.278380655626904
agent 18 在 57999 次迭代中的资源申请量为: 46.711864516741734
agent 19 在 57999 次迭代中的资源申请量为: 44.944698903213315
agent 20 在 57999 次迭代中的资源申请量为: 50.96604022350041
agent 21 在 57999 次迭代中的资源申请量为

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 56.3773885359945
agent 2 在 59999 次迭代中的资源申请量为: 56.20670471401895
agent 3 在 59999 次迭代中的资源申请量为: 60.86270576327959
agent 4 在 59999 次迭代中的资源申请量为: 38.05456524759511
agent 5 在 59999 次迭代中的资源申请量为: 57.75216512558106
agent 6 在 59999 次迭代中的资源申请量为: 55.53624349344336
agent 7 在 59999 次迭代中的资源申请量为: 65.85662660233905
agent 8 在 59999 次迭代中的资源申请量为: 46.50341640346253
agent 9 在 59999 次迭代中的资源申请量为: 65.85974500722061
agent 10 在 59999 次迭代中的资源申请量为: 54.400096430280875
agent 11 在 59999 次迭代中的资源申请量为: 65.67390625753639
agent 12 在 59999 次迭代中的资源申请量为: 38.545089748602464
agent 13 在 59999 次迭代中的资源申请量为: 49.88159271395787
agent 14 在 59999 次迭代中的资源申请量为: 41.50191258561869
agent 15 在 59999 次迭代中的资源申请量为: 42.64077943771322
agent 16 在 59999 次迭代中的资源申请量为: 49.418835153380385
agent 17 在 59999 次迭代中的资源申请量为: 50.27845361369593
agent 18 在 59999 次迭代中的资源申请量为: 46.7115047912937
agent 19 在 59999 次迭代中的资源申请量为: 44.944623484711926
agent 20 在 59999 次迭代中的资源申请量为: 50.966046448768594
agent 21 在 59999 次迭代中的资源申请量为: 7

In [24]:
np.savetxt('./data/01_IOS_q=4_agent_resource_quantity_Gaussian_attack_small.txt',agent_resource_quantity_robust4,fmt='%0.8f')#用户资源
np.savetxt('./data/01_IOS_q=4_average_cost_honest_Gaussian_attack_small.txt',average_cost_honest_robust4,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/01_IOS_q=4_average_resource_honest_Gaussian_attack_small.txt',average_resource_honest_robust4,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/01_IOS_q=4_dual_variable_Gaussian_attack_small.txt',dual_variable_robust4,fmt='%0.8f')#用户对偶变量